# eoAPI registration example
This notebook demonstrates how to register data on the workspace eoAPI instance. The main step is creation of STAC metadata information to pass it to the eoAPI STAC endpoint.
In order to show the full flow this notebooks also fetches some sample data and does following steps:
1. **Download**: Download file to mounted bucket path.
2. **Collection creation**: Create the STAC collection the file will be registered to.
3. **Metadata extraction**: Extract as much information as possible from source file to create STAC item.
4. **Ingest**: Post the resulting STAC ttem to the eoAPI STAC endpoint.

#### Environment Checks

In [1]:
# First some checks to see the environment is as expected
import sys
import os
import importlib.util

# List the packages to check
required = ["pystac", "rio_stac", "pystac_client", "requests", "rasterio"]
missing = [p for p in required if importlib.util.find_spec(p.replace('-','_')) is None]

if missing:
    print(f" ERROR: Missing packages: {', '.join(missing)}")
    print("It looks like the wrong Conda kernel is active. Please switch kernels and try again.")
    sys.exit("Execution stopped: Missing dependencies.")

# If check passes, proceed with imports
from datetime import datetime, timezone
from pystac import Collection, Extent, SpatialExtent, TemporalExtent, Link
from rio_stac import create_stac_item
import rasterio
import requests

print(" Environment ready.")

✅ Environment ready.


### Configuration
If using your own file adjust `FILE_PATH` to match its location and remove the download step.

In [2]:
STAC_API_URL = "http://eoapi-rw-stac:8080" 
COLLECTION_ID = "demo-collection"
COG_SOURCE_URL = "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/36/Q/WD/2020/7/S2A_36QWD_20200701_0_L2A/TCI.tif"
FILE_PATH = "./bucket/demo_data/TCI.tif"

os.makedirs(os.path.dirname(FILE_PATH), exist_ok=True)

### 1. Download

In [3]:
# First we download the example file and put it into the mounted bucket
# You can also use the File Browser to upload your data
# (should be used for files smaller then 1 GB)
if os.path.exists(FILE_PATH):
    print(f"File exists at {FILE_PATH}. Skipping.")
else:
    print(f"Downloading to {FILE_PATH}...")
    with requests.get(COG_SOURCE_URL, stream=True) as r:
        r.raise_for_status()
        with open(FILE_PATH, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

File exists at ./bucket/demo_data/TCI.tif. Skipping.


### 2. Collection creation

In [4]:
# Then we create the collection (only if it does not already exist)
check_url = f"{STAC_API_URL}/collections/{COLLECTION_ID}"
resp = requests.get(check_url)
if resp.status_code == 200:
    print(f"Collection '{COLLECTION_ID}' exists.")
else:
    print(f"Creating collection '{COLLECTION_ID}'...")
    coll = Collection(
        id=COLLECTION_ID,
        title="Demo collection",
        description="Demo COG collection",
        extent=Extent(SpatialExtent([[-180, -90, 180, 90]]), TemporalExtent([[datetime(2020,1,1,tzinfo=timezone.utc), None]])),
        license="CC-BY-4.0"
    )
    # Sending a POST request with the Collection json to create the collection
    requests.post(f"{STAC_API_URL}/collections", json=coll.to_dict()).raise_for_status()

Creating collection 'demo-collection'...


In [ ]:
# If we wanted to delete the collection we could use following command
# requests.delete(f"{STAC_API_URL}/collections/{COLLECTION_ID}")

### 3. Metadata Extraction

In [5]:
# We need a STAC item definition in order to register it into the collection
# see https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md for further details

# rio-stac provides a useful helper tool create_stac_item
# to extract metadata from the raster file and create a STAC item from it

# We need to define the connected bucket to this workspace
BUCKET = os.getenv("workspace_BUCKET")
ROOT_FILE_PATH = FILE_PATH.replace("./bucket", "")
# cloud location eoAPI TiTiler will access the data assets from
S3_URI = f"s3://{BUCKET}{ROOT_FILE_PATH}"

item = create_stac_item(
    source=FILE_PATH,
    asset_href=S3_URI,
    id=os.path.basename(ROOT_FILE_PATH).split('.')[0],
    collection=COLLECTION_ID,
    asset_name="data"
)

# Add preview link to allow rendering data in eoAPI GUI
eoapi_endpoint = os.getenv("workspace_RASTER_ENDPOINT")
xyz = f"{eoapi_endpoint}/collections/{COLLECTION_ID}/items/{item.id}/tiles/WebMercatorQuad/{{z}}/{{x}}/{{y}}?assets=data"
item.add_link(Link(rel="xyz", target=xyz, media_type="application/json"))


In [ ]:
# We can print the item by uncommenting if we want to have a look
# item.to_dict()

### 4. Ingestion

In [6]:
# Now that the item was created we can pass it to eoAPI for ingestion

url = f"{STAC_API_URL}/collections/{COLLECTION_ID}/items"
response = requests.post(url, json=item.to_dict())

if response.status_code in [200, 201]:
    print("Successfully posted STAC Item.")
else:
    print(f"Error {response.status_code}: {response.text}")

Successfully posted STAC Item.


In [7]:
# Update temporal and spatial extent based on ingested datasets
import requests

def update_stac(url, cid):
    # 1. Fetch all items (limited to 1000 for brevity)
    items = requests.get(f"{url}/collections/{cid}/items?limit=1000").json()['features']
    
    # 2. Extract and calculate (flatten coordinates and collect dates)
    coords = [pt for f in items for pt in f['geometry']['coordinates'][0]]
    dts = [f['properties']['datetime'] for f in items]
    
    # 3. Patch the collection
    ext = {
        "spatial": {"bbox": [[min(c[0] for c in coords), min(c[1] for c in coords), 
                             max(c[0] for c in coords), max(c[1] for c in coords)]]},
        "temporal": {"interval": [[min(dts), max(dts)]]}
    }
    
    r = requests.patch(f"{url}/collections/{cid}", json={"extent": ext})
    print("✅ Updated" if r.ok else f"❌ Error: {r.text}")

update_stac(STAC_API_URL, COLLECTION_ID)

✅ Updated


In [8]:
from IPython.display import Image, display

url = f"{eoapi_endpoint}/collections/demo-collection/items/TCI/preview?assets=data"
display(Image(url=url, width=500))